In [1]:
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from itertools import chain
from tqdm import tqdm
import pandas as pd





In [2]:
def get_pages_nums(driver):
    pages_len=[]
    try:
        
        last_page=driver.find_element(By.CLASS_NAME, 'pageNav-main')
        last_page_num=last_page.find_elements(By.TAG_NAME,'a')
        pages=[]
        for n in last_page_num:
            try:
                pages.append(int(n.text))
            except ValueError:
                continue
        last_page=max(pages)
        pages_len=[*range(1,last_page+1)]
    except NoSuchElementException:
           pass
    
    
    return pages_len

In [3]:
def get_forum_name(driver):
    title=''
    try:
        header_title=driver.find_element(By.CLASS_NAME, 'p-body-header')
        title=header_title.find_element(By.CLASS_NAME, 'p-title-value').text
    except NoSuchElementException:
        pass
    return title

In [4]:
def get_subjects_list(driver):
    links_title=[]
    subject_ids=[]
    
    
    try:
        
        body_subjects=driver.find_element(By.CLASS_NAME, 'structItemContainer-group.js-threadList')
        
        subjects_div=body_subjects.find_elements(By.TAG_NAME, 'div')
        #for sub in subjects:
            #link=sub.find_element(By.TAG_NAME, 'a').get_attribute('href')
            
            #title=sub.find_element(By.TAG_NAME, 'a').text
            #links_title.append((link,title))
        for sub_div in subjects_div:
            
            if sub_div.get_attribute('data-author')!=None:
                subject_class=sub_div.get_attribute('class')
                subject_id=subject_class.split(' ')[3].split('-')[2]
                subject_ids.append(subject_id)
                subject=sub_div.find_element(By.CLASS_NAME, 'structItem-title')
                link=subject.find_element(By.TAG_NAME, 'a').get_attribute('href')
                title=subject.find_element(By.TAG_NAME, 'a').text
                links_title.append((link,title,subject_id))
        
    except NoSuchElementException:
        links_title=[]
    return links_title
        
    
    

In [5]:
def get_user_id(driver):
    user_ids=[]
    user_id=None
    
    
    try:
        articles=driver.find_elements(By.TAG_NAME, 'article')
        for article in articles:
            if article.get_attribute('data-author')!=None:
                try:
                    user_id=article.find_element(By.CLASS_NAME, 'username').get_attribute('data-user-id')
                    #username=article.find_element(By.CLASS_NAME, 'username').text
                except NoSuchElementException:
                    user_id=''
                    #username=None
                user_ids.append(user_id)
    except NoSuchElementException:
        pass
            
    return user_ids
    

In [6]:
def get_username(driver):
    usernames=[]
    username=None
    
    
    try:
        articles=driver.find_elements(By.TAG_NAME, 'article')
        for article in articles:
            if article.get_attribute('data-author')!=None:
                try:
                    
                    username=article.find_element(By.CLASS_NAME, 'username').text
                except NoSuchElementException:
                    username=''
                    #username=None
                usernames.append(username)
    except NoSuchElementException:
        pass
    return usernames
    

In [7]:
def get_author_location(driver):
    author_locations=[]
    location=None
    extra=None
    try:
        articles=driver.find_elements(By.TAG_NAME, 'article')
        for article in articles:
            if article.get_attribute('data-author')!=None:
                try:
                    extras=article.find_element(By.CLASS_NAME, 'message-userExtras')
                    extras_details=extras.find_elements(By.TAG_NAME, 'dl')
                    for ex in extras_details:
                        extra=ex.find_element(By.TAG_NAME, 'dt').text
            
                        if extra=="Location":
                            location=ex.find_element(By.TAG_NAME, 'dd').text
                except NoSuchElementException:
                    location=''
                author_locations.append(location)
                    
                
                
            
    except NoSuchElementException:
        pass
    return author_locations
                
        
    
    

In [8]:
def get_date(driver):
    dates=[]
    date=None
    try:
        articles=driver.find_elements(By.TAG_NAME, 'article')
        for article in articles:
            if article.get_attribute('data-author')!=None:
                try:
                    date_li=article.find_element(By.CLASS_NAME, 'u-concealed')
                    date=date_li.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
                except NoSuchElementException:
                    date=''
                dates.append(date)
    except NoSuchElementException:
        pass
    return dates
#//*[@id="js-post-3218790"]/div/div[2]/div/header/ul[1]/li/a/time
#//*[@id="js-post-3218790"]/div/div[2]/div/header/ul[1]/li
   #//*[@id="js-post-3218787"]/div/div[2]/div/header/ul[1]/li     
    #//*[@id="js-post-3218787"]/div/div[2]/div/header/ul[1]/li/a/time
 #js-post-3218787 > div > div.message-cell.message-cell--main > div > header > ul.message-attribution-main.listInline > li

In [9]:
def get_text(driver):
    texts=[]
    text=None
    try:
        articles=driver.find_elements(By.TAG_NAME, 'article')
        for article in articles:
            if article.get_attribute('data-author')!=None:
                try:
                    text=article.find_element(By.CLASS_NAME, 'message-body.js-selectToQuote').text.replace('\n',' ').strip()
                    
                    
                except NoSuchElementException:
                    text=''
                texts.append(text)
    except NoSuchElementException:
        pass
    return texts
        
    
    

In [10]:
def get_discussion(forum_name,driver, subject_id, subject, page):
   
    
    
    author_ids= get_user_id(driver)
    author_names=get_username(driver)
    author_locations=get_author_location(driver)
    dates=get_date(driver)
    texts=get_text(driver)
    forums=[forum_name]*len(dates)
    subject_ids=[subject_id]*len(dates)
    subjects=[subject]*len(dates)
    author_or_replier=[]
    
    post_author_id=author_ids[0]
    init_post=[]
    if page==1:
        for i in range(len(dates)):
            if i==0:
                init_post.append(True)
            else:
                init_post.append(False)
    else:
        for i in range(len(dates)):
            init_post.append(False)
    
    for aut_id in author_ids:
        if aut_id==post_author_id:
            author_or_replier.append('author')
        else:
            author_or_replier.append('replier')
    
    
    return list(zip(forums, subject_ids, subjects,dates, author_or_replier, init_post,author_ids,author_locations, author_names, texts))
    
            
    
    
    

In [11]:
def discussion_to_df(driver,forum_name, dis_dir):
    subjects=get_subjects_list(driver)
        
    for s in subjects:
        sub_url=s[0]
        title=s[1]
        subject_id=s[2]
        
        driver.get(sub_url)
        WebDriverWait(driver, 5).until(lambda d: d.find_element(By.CLASS_NAME, 'p-body-pageContent'))
        author_location=get_author_location(driver)
        london_loc=None
        london_loc_lower=None
        pattern_condition=False
        if (len(author_location)>0) & (author_location[0]!=None):
            pattern_to_match=['\w+\s+London','London', '^London', ',London$', '\s+London$', '\s+London+\s', '^London+\s', '^London+\s+\w', '^London,']
            
            for pat in pattern_to_match:
                if re.match(pat, author_location[0], re.IGNORECASE)!=None:
                    
                    
                    pattern_condition=True
                    break
                    
                
            
           
            
        if pattern_condition==False:
            continue
        else:
            
            sub_pages=get_pages_nums(driver)
            if len(sub_pages)==0:
                
                discussion=get_discussion(forum_name, driver, subject_id,title,1)
                dis_df=pd.DataFrame(discussion, columns=['forums', 'subject_id', 'subject','date', 'author_or_replier', 'init_post','author_id','author_locations', 'author_name', 'texts'])
                dis_df.to_csv(dis_dir+str(subject_id)+'.csv')
            else:
                discussion_sub=[]
                for sub_p in sub_pages:
                    
                    if sub_p==1:
                        discussion=get_discussion(forum_name, driver, subject_id,title, sub_p)
                        discussion_sub.append(discussion)
                        
                    else:
                        
                        
                        url_page=sub_url+'page-'+str(sub_p)
                        driver.get(url_page)
                        WebDriverWait(driver, 5).until(lambda d: d.find_element(By.CLASS_NAME, 'p-body-pageContent'))
                        discussion=get_discussion(forum_name, driver, subject_id,title, sub_p)
                        discussion_sub.append(discussion)
                    
                         
                dis_df=pd.DataFrame(list(chain(*discussion_sub)), columns=['forums', 'subject_id', 'subject','date', 'author_or_replier', 'init_post','author_id','author_locations', 'author_name', 'texts'])
                dis_df.to_csv(dis_dir+str(subject_id)+'.csv')
    

In [12]:
anxiety_forum='https://www.mentalhealthforum.net/forum/forums/anxiety-forum.365/'
anxiety_folder='/home/slawomir/Portfolio/omdena_self_harm_pred/mentforum/anxiety/'
depression_forum='https://www.mentalhealthforum.net/forum/forums/depression-forum.366/'
depression_folder='/home/slawomir/Portfolio/omdena_self_harm_pred/mentforum/depression/'
eating_disorder_forum='https://www.mentalhealthforum.net/forum/forums/eating-disorders-forum.32/'
eating_disorder_folder='/home/slawomir/Portfolio/omdena_self_harm_pred/mentforum/eating_disorder/'
health_anxiety_forum='https://www.mentalhealthforum.net/forum/forums/health-anxiety-forum-illness-anxiety-forum.398/'
health_anxiety_folder='/home/slawomir/Portfolio/omdena_self_harm_pred/mentforum/health_anxiety/'

In [13]:
#forums_tup=[(anxiety_forum)]

In [14]:
from tqdm import tqdm


def get_forum(forum_url,forum_dir):
    driver = webdriver.Chrome(executable_path='/home/slawomir/chrome/chromedriver')
    
    
    driver.get(forum_url)
    forum_name=get_forum_name(driver)
    WebDriverWait(driver, 5).until(lambda d: d.find_element(By.CLASS_NAME, 'p-body-pageContent'))
    pages=get_pages_nums(driver)
    
    for p in tqdm(pages[81:]):


        if p==1:
            discussion_to_df(driver,forum_name,forum_dir)




        else:
            forum_sub_url=forum_url+'page-'+str(p)
            driver.get(forum_sub_url)
            WebDriverWait(driver, 5).until(lambda d: d.find_element(By.CLASS_NAME, 'p-body-pageContent'))
            discussion_to_df(driver,forum_name,forum_dir)









                    
                    
                    
    


In [15]:
forum_url=eating_disorder_forum
dis_dir=eating_disorder_folder

get_forum(forum_url, dis_dir)

/tmp/ipykernel_4003/366916643.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/home/slawomir/chrome/chromedriver')
100%|███████████████████████████████████████████| 26/26 [34:28<00:00, 79.57s/it]


In [16]:
30/107

0.2803738317757009